### Running on a GPU

To speed things up further Theano can [run your code on a GPU](http://deeplearning.net/software/theano/tutorial/using_gpu.html). Theano relies on Nvidia's CUDA platform, so yo need a [CUDA-enabled graphics card](http://docs.nvidia.com/cuda/cuda-getting-started-guide-for-mac-os-x/#cuda-enabled-gpu). If you don't have one (for example, if you're on a Macbook with an integrated GPU) you could start up a [GPU-optimized Amazon EC2 instance](https://aws.amazon.com/ec2/instance-types/#gpu) and try things there. To use the GPU with Theano you must have the CUDA toolkit installed and [set a few environment variables](http://deeplearning.net/software/theano/install.html#gpu-linux).

While any Theano code will run just fine on the GPU you will most likely need to spend some time optimizing it to get optimal performance. Running code that isn't optimized for the GPU may actually be slower than using the CPU due to data transfer to and from the GPU.

Here are the some things you need to consider when you want to use a GPU:

- The default floating point data type is `float64`, but in order to use the GPU you must use `float32`. There are several ways to do this in Theano. You can either use the [fulled typed tensor constructors](http://deeplearning.net/software/theano/library/tensor/basic.html#all-fully-typed-constructors) or set  `theano.config.floatX` to `float32` and use the [simple tensor constructors](http://deeplearning.net/software/theano/library/tensor/basic.html#constructors-with-optional-dtype).
- Shared variables must be initialized with values of type `float32` in order to be stored on the GPU. This means that when you create a shared variable from a numpy array you must initialize the array with the `dtype=float32` argument, or cast the value using the `astype` function. See the [numpy data type documentation](http://docs.scipy.org/doc/numpy/user/basics.types.html) for more details.
- Be very careful with data transfers to and from the GPU. Ideally you want all your data on the GPU. Ways to achieve this include initializing commonly uses data as shared variables with a `float32` data type, and to [avoid copying return values](http://deeplearning.net/software/theano/tutorial/using_gpu.html#returning-a-handle-to-device-allocated-data) back to the host using the `gpu_from_host` function. Theano includes [profiling capabilities](http://deeplearning.net/software/theano/tutorial/profiling.html) that allow you to debug how much time you are spending on transfers.

With all that in mind, let's modify our code to run well on a GPU. I'll highlight the most important changes we made. The full code is available [here](https://github.com/dennybritz/nn-theano).

In [5]:
import numpy as np
import sklearn
import sklearn.datasets
import theano
import theano.tensor as T
import time

# Use float32 as the default float data type
theano.config.floatX = 'float32'

First, to get significant speedup from a GPU we should increase the size of our data. In that case the time will be dominated by our matrix multiplication and we get the most out of parallelization.

In [13]:
train_X, train_y = sklearn.datasets.make_moons(5000, noise=0.20)
nn_hdim = 1000

To force the storage of our data on the GPU we use shared variables of type `float32`. Because we are constantly using the input data `X` and `y` to perform gradient updates we store those on the GPU as well:

In [14]:
# These float32 values are initialized on the GPU
X = theano.shared(train_X.astype('float32'), name='X')
y = theano.shared(train_y_onehot.astype('float32'), name='y')
W1 = theano.shared(np.random.randn(nn_input_dim, nn_hdim).astype('float32'), name='W1')
b1 = theano.shared(np.zeros(nn_hdim).astype('float32'), name='b1')
W2 = theano.shared(np.random.randn(nn_hdim, nn_output_dim).astype('float32'), name='W2')
b2 = theano.shared(np.zeros(nn_output_dim).astype('float32'), name='b2')

We also modify our functions and remove inputs since the X and y variables are now  stored as shared variables on the GPU. We don't need to pass them to the functions anymore.

In [15]:
gradient_step = theano.function(
    inputs=[],
    updates=((W2, W2 - epsilon * dW2),
             (W1, W1 - epsilon * dW1),
             (b2, b2 - epsilon * db2),
             (b1, b1 - epsilon * db1)))

### Evaluation

I tesed the code on a `g2.2xlarge` AWS EC2 instance. Running one `gradient_step()` on the CPU took around 250ms. Running it on the GPU with `THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32` took 7.5ms. That's a 30x speedup, and if our dataset or parameter space was larger we probably would have seen an even more significant speedup.

In [12]:
%timeit gradient_step()

The slowest run took 14.01 times longer than the fastest. This could mean that an intermediate result is being cached 
10000 loops, best of 3: 113 µs per loop


### Full GPU-optimzied code

In [18]:
# Generate a dataset
np.random.seed(0)
train_X, train_y = sklearn.datasets.make_moons(5000, noise=0.20)
train_y_onehot = np.eye(2)[train_y]

# Size definitions
num_examples = len(train_X) # training set size
nn_input_dim = 2 # input layer dimensionality
nn_output_dim = 2 # output layer dimensionality
nn_hdim = 1000 # hiden layer dimensionality

# Gradient descent parameters (I picked these by hand)
epsilon = np.float32(0.01) # learning rate for gradient descent
reg_lambda = np.float32(0.01) # regularization strength 

# GPU NOTE: Conversion to float32 to store them on the GPU!
X = theano.shared(train_X.astype('float32')) # initialized on the GPU
y = theano.shared(train_y_onehot.astype('float32'))

# GPU NOTE: Conversion to float32 to store them on the GPU!
W1 = theano.shared(np.random.randn(nn_input_dim, nn_hdim).astype('float32'), name='W1')
b1 = theano.shared(np.zeros(nn_hdim).astype('float32'), name='b1')
W2 = theano.shared(np.random.randn(nn_hdim, nn_output_dim).astype('float32'), name='W2')
b2 = theano.shared(np.zeros(nn_output_dim).astype('float32'), name='b2')

# Forward propagation
z1 = X.dot(W1) + b1
a1 = T.tanh(z1)
z2 = a1.dot(W2) + b2
y_hat = T.nnet.softmax(z2)

# The regularization term (optional)
loss_reg = 1./num_examples * reg_lambda/2 * (T.sum(T.sqr(W1)) + T.sum(T.sqr(W2))) 
# the loss function we want to optimize
loss = T.nnet.categorical_crossentropy(y_hat, y).mean() + loss_reg
# Returns a class prediction
prediction = T.argmax(y_hat, axis=1)

# Gradients
dW2 = T.grad(loss, W2)
db2 = T.grad(loss, b2)
dW1 = T.grad(loss, W1)
db1 = T.grad(loss, b1)

# Note that we removed the input values because we will always use the same shared variable
# GPU NOTE: Removed the input values to avoid copying data to the GPU.
forward_prop = theano.function([], y_hat)
calculate_loss = theano.function([], loss)
predict = theano.function([], prediction)

# GPU NOTE: Removed the input values to avoid copying data to the GPU.
gradient_step = theano.function(
    [],
    # profile=True,
    updates=((W2, W2 - epsilon * dW2),
             (W1, W1 - epsilon * dW1),
             (b2, b2 - epsilon * db2),
             (b1, b1 - epsilon * db1)))

def build_model(num_passes=20000, print_loss=False):
    # Re-Initialize the parameters to random values. We need to learn these.
    np.random.seed(0)
    # GPU NOTE: Conversion to float32 to store them on the GPU!
    W1.set_value((np.random.randn(nn_input_dim, nn_hdim) / np.sqrt(nn_input_dim)).astype('float32'))
    b1.set_value(np.zeros(nn_hdim).astype('float32'))
    W2.set_value((np.random.randn(nn_hdim, nn_output_dim) / np.sqrt(nn_hdim)).astype('float32'))
    b2.set_value(np.zeros(nn_output_dim).astype('float32'))
    
    # Gradient descent. For each batch...
    for i in xrange(0, num_passes):
        # This will update our parameters W2, b2, W1 and b1!
        gradient_step()
        
        # Optionally print the loss.
        # This is expensive because it uses the whole dataset, so we don't want to do it too often.
        if print_loss and i % 1000 == 0:
            print "Loss after iteration %i: %f" %(i, calculate_loss())



In [ ]:
# Profiling
# theano.config.profile = True
# theano.config.profile_memory = True
# gradient_step()
# theano.printing.debugprint(gradient_step) 
# print gradient_step.profile.summary()

In [17]:
%timeit gradient_step()

# start = time.time()
# build_model()
# print "Total training time:", time.time()-start, "sec"

1 loops, best of 3: 184 ms per loop
